In [ ]:
import pydicom
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import threshold_otsu
from skimage.transform import resize
from collections import defaultdict
import pandas as pd
import os

% matplotlib inline

In [ ]:
class ROI:
    
    def __init__(self, filename, border, character, flipped):
        dataset = pydicom.dcmread(filename)
        a = dataset.pixel_array
        
        if a.shape[0] < 1000:
            fl = list(filename)
            if fl[-5] == '0':
                fl[-5] = '1'
                filename = "".join(fl)
            else:
                fl[-5] = '0'
                filename = ''.join(fl)
                
        dataset = pydicom.dcmread(filename)
        a = dataset.pixel_array
        '''
        if "LEFT" in filename:
            self.mode = 'left'
        else:
            self.mode = 'right'
            a = a[:,::-1]
        '''
        if flipped:
            a = a[:, ::-1]
        
        self.filename = filename
        self.image = resize(a[:, :border], (227,227))
        self.character = character
        
    def plot_contour(self):
        if self.character == 'benign':
            plt.contour(self.image, cmap = 'plasma', linewidths = 0.5 )
        else:
            plt.contour(self.image, cmap = 'winter', linewidths = 0.5 )


class Photo:
    
    def find_border(self, x):
    
        '''
        arr = np.sum(x[1000:4000, :], axis = 0)
        i = 0
        while(arr[i] > 0):
            i += 1
        return i
        '''
        arr = np.sum(x[1000:4000, :], axis = 1)
        #plt.plot(range(len(arr)), arr)
        am = np.argmax(arr)
        #print(am)
        
        for i in range(x.shape[1] - 100 , 0 , -1):
            if x[1000 + am, i] > 0:
                return i
    
    def plot_image(self):
        
        plt.imshow(self.image, cmap = 'gray')
        
        for roi in  self.m_rois:
            roi.plot_contour()
            
        for roi in  self.b_rois:
            roi.plot_contour()
            
        plt.show()
            
    
    def __init__(self, filename, filenames_ROI_m, filenames_ROI_b):
        dataset = pydicom.dcmread(filename)
        a = dataset.pixel_array
        
        '''
        if "LEFT" in filename:
            self.mode = 'left'
        else:
            self.mode = 'right'
            a = a[:,::-1]
        '''

            
        self.filename = filename
        self.threshold = threshold_otsu(a)
        
        self.flipped = False
        
        temp = a > self.threshold
        if np.sum(temp[:, :temp.shape[1] // 2]) > np.sum(temp[:, temp.shape[1] // 2:]):
            self.mode = 'left'
        else:
            self.mode = 'right'
            a = a[:,::-1]
            self.flipped = True
        
        self.border = self.find_border(a > self.threshold)

        
        
        #plt.imshow(a > self.threshold, cmap = 'gray')
        
        self.image = resize(a[:, :self.border], (227,227))
        
        self.m_rois = []
        self.b_rois = []
        
        for fn in filenames_ROI_m:
            self.m_rois.append(ROI(fn, self.border, 'malignant', self.flipped))
        
        for fn in filenames_ROI_b:
            self.b_rois.append(ROI(fn, self.border, 'benign', self.flipped))
        
        

In [ ]:
s = 'adasdsd'
s[0] = 'x'

In [ ]:
filename = "CBIS-DDSM/Mass-Test_P_00016_LEFT_CC/10-04-2016-DDSM-30104/1-full mammogram images-14172/000000.dcm"
f1 = "CBIS-DDSM/Mass-Test_P_00016_LEFT_CC_1/10-04-2016-DDSM-09887/1-cropped images-26184/000001.dcm"
filenames_ROI_m = [f1]
filenames_ROI_b = []

ph = Photo(filename, filenames_ROI_m, filenames_ROI_b)

In [ ]:
ph.plot_image()

In [ ]:
import pickle

def cut_name(x):
    fst = x.find("/")
    scnd = fst + x[fst + 1:].find('/') + 1
    return x[fst+1:scnd]

filehandler = open('Data/' + cut_name(ph.filename), 'wb') 
pickle.dump(ph, filehandler)

In [ ]:
with open('Data/' + cut_name(ph.filename), 'rb') as filehandler:
    ph = pickle.load(filehandler)

In [ ]:
ph

In [ ]:
'asdaasd'.find("d")

In [ ]:
from collections import defaultdict

def parse_name(filename):
    ind = 10 + filename[10:].find('/')
    sp = filename[10:ind]

    if '0' <= sp[-1] <= '9':
        return sp[:-2], sp, filename + '/000001.dcm' , 0
    else:
        return sp, sp,  filename + '/000000.dcm', 1
    
    

In [ ]:
originals_mass = dict()
rois_mass = defaultdict(dict)

for root, dirs, files in os.walk("CBIS-DDSM/"):  
    for filename in files:
        if 'Mass' in root:
            entity, name,  addr, reg = parse_name(root.replace('\\', '/'))
            if reg == 0:
                rois_mass[entity][name] = addr
            else:
                originals_mass[entity] = addr

In [ ]:
#calc_test = pd.read_csv("calc_case_description_test_set.csv")
#calc_train = pd.read_csv("calc_case_description_train_set.csv")
mass_test = pd.read_csv("mass_case_description_test_set.csv")
mass_train = pd.read_csv("mass_case_description_train_set.csv")

mass_dict = {}

for pat_id, direct, view, abn_id, pathology in  mass_test.loc[:,("patient_id", 'left or right breast',\
                                                                'image view', 'abnormality id', 'pathology')].values:
    mass_dict['Mass-Test_' + pat_id + '_' + direct + '_' + view + '_' + str(abn_id)] = pathology


for pat_id, direct, view, abn_id, pathology in  mass_train.loc[:,("patient_id", 'left or right breast',\
                                                                'image view', 'abnormality id', 'pathology')].values:
    if pathology != 'MALIGNANT':
        pathology = 'BENIGN'
    mass_dict['Mass-Training_' + pat_id + '_' + direct + '_' + view + '_' + str(abn_id)] = pathology


In [ ]:
rois_mass['CBIS-DDSM/Mass-Test_P_00037_RIGHT_CC/10-04-2016-DDSM-74559/1-full mammogram images-82316/000000.dcm']

In [ ]:
MASS = []
for entity in originals_mass.keys():
    mal = []
    ben = []
    rois = rois_mass[entity]
    for roi in rois.keys():
        if mass_dict[roi] == 'BENIGN':
            ben.append(rois[roi])
        else:
            try:
                mal.append(rois[roi])
            except Exception as e:
                print('*********************************************')
                print(rois[roi])
                print(e)
    try:
        ph = Photo(originals_mass[entity], mal, ben)
        print(ph.filename)
        print(ph.mode)
        ph.plot_image()
        
        with open('Data/' + cut_name(ph.filename), 'wb') as filehandler:
            pickle.dump(ph, filehandler)
        #MASS.append(ph)
        #stop = int(input("Stop?"))
        #if stop == 1:
        #    break
        
    except Exception as e:
        print('********************************************')
        print(rois[roi])
        print(e)

In [ ]:
len(MASS)

In [ ]:
MASS[0].plot_image()

In [ ]:
set(mass_test.pathology)

In [ ]:
ph

In [ ]:
from skimage.filters import threshold_otsu
from skimage.transform import resize
import os

In [ ]:
val = threshold_otsu(a / 65535)

In [ ]:



w = list(os.walk("CBIS-DDSM/"))

In [ ]:
import re
[m.start() for m in re.finditer('\', 'test test test test')]

In [ ]:


val = threshold_otsu(a)
plt.imshow(a > val, cmap = 'gray')
border = find_border(a > val, mode = 'right')
plt.plot([border for i in range(4600)], [i for i in range(4600)], c = 'red')
plt.show()


In [ ]:
plt.imshow((a > val)[:,::-1], cmap = 'gray')

In [ ]:
np.min((a > val).sum(axis = 0))

In [ ]:
arr = np.sum(a > val, axis = 1)
plt.plot(range(len(arr)), arr)
plt.show()

In [ ]:
(np.sum(a > val, axis = 1) )[1000:1500]

In [ ]:
plt.imshow(resize(a[:, :border], (227,227)), cmap = 'gray')

In [ ]:
resize(a[:, :border], (227,227)).shape

In [ ]:
np.sum(a, axis = 0).shape

In [ ]:
np.max(a / 65535)

In [ ]:
filehandler.close()

In [ ]:
dataset1 = pydicom.dcmread("CBIS-DDSM/Calc-Test_P_00038_RIGHT_CC/08-29-2017-DDSM-28468/1-full mammogram images-63304/000000.dcm")

In [ ]:
dataset2 = pydicom.dcmread("CBIS-DDSM/Calc-Test_P_00038_RIGHT_CC_1/08-29-2017-DDSM-83105/1-ROI mask images-37851/000001.dcm")

In [ ]:
a1 = dataset1.pixel_array
a2 = dataset2.pixel_array

In [ ]:
plt.imshow(resize(a1 / 66000, (227,227)), cmap = 'gray')
#plt.imshow(, cmap = 'Blues', alpha = 0.15)
plt.contour(resize( a2 / 66000, (227,227)), cmap = 'winter', linewidths = 0.5)

In [ ]:
plt.contour(a2)

In [ ]:
#plt.imshow(resize(a1, (227,227)), cmap = 'gray')
model = xception.Xception(include_top=False, weights='imagenet', input_shape=(227,227,3) , pooling='avg')

In [ ]:
import keras
from keras.applications import  xception#mobilenet_v2,
from keras.layers import Input, Convolution2D, Lambda, Flatten
import tensorflow as tf
from keras.optimizers import Adam
#model = xception.Xception(include_top=False, weights='imagenet', input_shape=(227,227,3) , pooling='avg')


def create_model(m, lambd_reg, learning_rate):
    inp = m.get_layer(name = 'input_12').output
    out = m.get_layer(name = 'block14_sepconv2_act').output
    m1 = Model(inputs = inp, outputs = out)

    inter = m1.output
    conv = Convolution2D(1, (1,1), name = 'Conv_Sigmoid', activation = 'sigmoid')
    flat = Flatten(name = 'Flatten')
    lda = Lambda(lambda x: tf.contrib.framework.sort(x, axis=-1, direction = 'DESCENDING'), name="Sort")

    out = lda(flat(conv(inter)))
    
    m2 =  Model(inputs = inp, outputs = out)
    
    for layer in m2.layers:
        layer.kernel_regularizer = regularizers.l2(lambd_reg)
        layer.bias_regularizer = regularizers.l2(lambd_reg)
    
    adam = Adam(learning_rate)
    m2.compile(optimizer = adam, loss = keras.losses.categorical_crossentropy)
    return m2


class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, labels, batch_size=32, dim=(227,227), n_channels=3,
                 n_classes=10, shuffle=True, TOP_K = 5, OUT_SIZE = 49):
        'Initialization'
        
        self.dim = dim # ?-1
        
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        #self.n_classes = n_classes ?-2
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        y = np.empty((self.batch_size, OUT_SIZE), dtype='int32')
        
        photo = 0
        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            
            with open('Data/' + cut_name(ID), 'rb') as filehandler:
                photo = pickle.load(filehandler)
                
            #photo = np.load('data/' + ID + '.npy')
            img = photo.image
            
            X[i,] = np.repeat(img.reshape(*self.dim, 1), 3, axis=2)   
            
            yy = np.zeros(OUT_SIZE, dtype = 'int32')
            
            if len(photo.m_rois) == 0:
                y[i,] = yy
            else:
                yy[:TOP_K] = 1
                y[i,] = yy
                

        return X, y


In [ ]:
mm = create_model(model, lambd_reg = 1e-5, learning_rate = 5e-5)



In [ ]:
np.array(['1', '2'])

In [ ]:

from keras import regularizers
mm.layers[5].kernel_regularizer

In [ ]:
mm.layers[4].kernel_regularizer

In [ ]:
mm.summary()

In [ ]:
mm.output

In [ ]:
print(tf.__version__)

In [ ]:
t = tf.constant([[[1, 1, 1], [2, 2, 2]],
                 [[3, 3, 3], [4, 4, 4]],
                 [[5, 5, 5], [6, 6, 6]]])
tf.slice(t, [0,0,1], [1,1,2])

In [ ]:
model.summary()

In [ ]:
conv = Convolution2D(1, (7,7))

In [ ]:
m1.summary()

In [ ]:
#inp = tf.placeholder(shape=(None, 227,227,3), dtype = 'float32')

inter = m1.output

conv = Convolution2D(1, (1,1))

out = conv(inter)

In [ ]:
out

In [ ]:
#,input_tensor=Input((227,227,1))
inp = tf.placeholder(shape=(227,227,3), dtype = 'float32')
model = xception.Xception(include_top=False, weights='imagenet',input_tensor=Input((227,227,3)),input_shape=(227,227,3) , pooling='avg')
model.layers.pop()
#model = mobilenet_v2.MobileNetV2(input_shape=(224,224,3), alpha=1.3, depth_multiplier=1, include_top=False, weights='imagenet', input_tensor=None, pooling= 'avg')

In [ ]:
model.summary()

In [ ]:
from keras.models import Model

In [ ]:
import tensorflow as tf
from keras.layers import Convolution2D

In [ ]:
model.add(Convolution2D(1, (7,7)))

In [ ]:
from keras.models import Sequential
inp =  tf.placeholder(shape=(None, 227,227,3), dtype = 'float32')
model(inp)

In [ ]:
inp =  tf.placeholder(shape=(227,227,3, None), dtype = 'float32')
m1 = model.layers[1](inp)
for layer in model.layers[2:]:
    m1 = layer(m1)

In [ ]:
m = Model(inputs = inp, outputs = model)

In [ ]:
img = ph.image

In [ ]:
np.stack((img,)*3, axis=-1)

In [ ]:
from keras.models import Sequential, Model

